### Overview: Single-Model Tuning Pipeline

- This notebook outlines the development workflow of a **Random Forest–based machine learning pipeline** for iron deficiency identification. It includes the full process of training, feature selection, and hyperparameter tuning.

- Due to internal data governance policies, we are unable to share the original patient-level datasets. As a result, this notebook is **not executable out-of-the-box**. This notebook is intended to illustrate the overall modeling workflow and the logic of each step to support transparency and potential reproducibility when appropriate data access is granted.

---


#### Workflow Summary

1. **Import and Environment Setup**  
   Load all required libraries and project paths.

2. **Data Preparation**  
   Paths are initialized to read in pre-split training and test datasets for internal and external validation (An-Nan Hospital and Wei-Gong Memorial Hospital).

3. **Cross-Validation with a Single Model (Random Forest)**  
   Perform stratified K-fold training on a fixed variable combination, storing fold-wise predictions and metrics.

4. **Fold Selection**  
   Identify the best-performing fold based on AUROC and other metrics for subsequent feature analysis.

5. **Feature Importance Extraction**  
   Use the selected Random Forest model to compute importance scores and sort features accordingly.

6. **Recursive Feature Elimination**  
   Iteratively remove the least important features to evaluate performance impact and identify the optimal feature subset.

7. **Model Retraining with Top Features**  
   Re-train the model using only the best N features determined in the previous step.

8. **Hyperparameter Tuning**  
   Perform `RandomizedSearchCV` to optimize Random Forest hyperparameters for better performance

9. **Final Evaluation**  
   Train the final model using optimized settings and selected features, and evaluate performance across all cohorts using AUROC, AUPRC, and Brier score.

---




### **Import Library**

In [ ]:

import os
import sys
import time

# ============================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from joblib import dump, load
from scipy.stats import randint
from sklearn.metrics import (
    make_scorer, 
    roc_auc_score, 
    f1_score, 
    brier_score_loss
)
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline

# ============================

sys.path.append('../src') 
from config import data_dir, cache_dir
from model_utils import (
    get_chosen_columns,
    evaluate_model,
    imputer,
    scaling
)


### **Settings**

In [ ]:
random_seed= 190   
threshold= 0.5   # for binary threshold for positive prediction
K=5              # for k-fold cross validation

female_ferritin_threshold= 11
male_ferritin_threshold= 23.9
TIBC_threshold= 450

weight_for_0 = 1 # for iron deficiency negative samples
weight_for_1 = 8 # for iron deficiency positive samples
    
impute_method= 'zero'

### **Loading data**

In [ ]:
# Load training and test data (internal validation) 
data_train = pd.read_csv(
    os.path.join(data_dir, 'CMUH_train_set.csv'),
    index_col=0
)

data_test = pd.read_csv(
    os.path.join(data_dir, 'CMUH_valid_set.csv'),
    index_col=0
)

# Load external validation datasets from AN and WMH hospitals
data_ANH = pd.read_csv(
    os.path.join(data_dir, 'ANH.csv'),
    index_col=0
)

data_WMH = pd.read_csv(
    os.path.join(data_dir, 'WMH.csv'),
    index_col=0
)

In [ ]:
# Recalculate and summarize the proportion and counts of positive cases in each dataset
positive_ratios_counts = {
    'data_train': (data_train['Label'].mean() * 100, data_train['Label'].sum(), len(data_train) - data_train['Label'].sum()),
    'data_test': (data_test['Label'].mean() * 100, data_test['Label'].sum(), len(data_test) - data_test['Label'].sum()),
    'data_ANH': (data_ANH['Label'].mean() * 100, data_ANH['Label'].sum(), len(data_ANH) - data_ANH['Label'].sum()),
    'data_WMH': (data_WMH['Label'].mean() * 100, data_WMH['Label'].sum(), len(data_WMH) - data_WMH['Label'].sum()),
}

# Display the results: percentage of positives, and counts of positive/negative cases
for dataset, (percentage, positive, negative) in positive_ratios_counts.items():
    print(f"{dataset}: {percentage:.2f}% ({int(positive)} positive cases, {int(negative)} negative cases)")

data_train: 18.83% (1026 positive cases, 4423 negative cases)
data_test: 15.99% (273 positive cases, 1434 negative cases)
data_AN: 19.27% (264 positive cases, 1106 negative cases)
data_WK: 19.62% (249 positive cases, 1020 negative cases)


### **Cross validation with single model**
- model: Random forest
- Impute method: Zero
- variables set: 'CBC','CPD','basic'
  

In [ ]:
# Define feature groups and retrieve selected columns
var = ['CBC', 'CPD', 'basic']
chosen_col = get_chosen_columns(var)

# Define imputation strategy (only testing 'zero' imputation here)
impute_groups = ['zero']

# Assign features and labels from datasets
x = data_train.loc[:, chosen_col] 
y = data_train['Label']


In [ ]:
# Initialize cross-validation splitter
skf = StratifiedKFold(n_splits=K, random_state=random_seed, shuffle=True)

# Initialize storage for performance metrics
metrics_columns = ['AUROC', 'AUPRC', 'accuracy', 'F1_score', 'recall', 'specificity', 'precision', 'NPV']
outcome_total = pd.DataFrame({col: pd.Series(dtype='float') for col in metrics_columns})

# Begin cross-validation loop
fold = 0
for train_index, val_index in skf.split(x, y):
    fold += 1
    
    # Save indices to file for reproducibility
    dump(train_index, f'{cache_dir}\\train_index_{fold}.joblib')
    dump(val_index, f'{cache_dir}\\val_index_{fold}.joblib')

    # Split data into training and validation
    x_train_cv, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train_cv, y_val = y.iloc[train_index], y.iloc[val_index]

    # Compute sample weights for training
    weights_train = np.array([weight_for_0 if label == 0 else weight_for_1 for label in y_train_cv])

    # Imputation and scaling (train and apply same to val/test/external sets)
    x_train_imp, m_mean = imputer(x_train_cv, impute_method, train=True)
    x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

    x_val_imp = imputer(x_val, impute_method, False, m_mean)
    x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)

    # Train random forest model
    model = RandomForestClassifier(random_state=random_seed)
    model.fit(x_train_scale, y_train_cv, sample_weight=weights_train)

    # Inference for all datasets
    prob_val = model.predict_proba(x_val_scale)[:, 1]

    # Evaluate performance
    con_matrix_val, outcome, pred = evaluate_model(prob_val, y_val, threshold)

    # Append fold performance results
    outcome_total = pd.concat([outcome_total, outcome], ignore_index=True)

# Print full performance tables
print(outcome_total)


### **Pick up the Best-Performing Cross-Validation Fold**

In [ ]:
# Select specific fold index according to result in training set
i = 3  
print(var)

# Define target variable
y = data_train['Label']

# Extract features for all datasets
x = data_train.loc[:, chosen_col] 

# Load saved indices for the selected fold
train_index = load(f'{cache_dir}\\train_index_{i}.joblib')
val_index = load(f'{cache_dir}\\val_index_{i}.joblib')

# Split into training and validation sets
x_train, x_val = x.iloc[train_index], x.iloc[val_index]
y_train, y_val = y.iloc[train_index], y.iloc[val_index]

# Imputation and scaling
x_train_imp, m_mean = imputer(x_train, impute_method, train=True)
x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

x_val_imp = imputer(x_val, impute_method, False, m_mean)
x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)

# Compute sample weights for training
weights_train = np.array([weight_for_0 if label == 0 else weight_for_1 for label in y_train])

# Train model on selected fold
model = RandomForestClassifier(random_state=random_seed)
model.fit(x_train_scale, y_train, sample_weight=weights_train)

# Predict probabilities
prob_val = model.predict_proba(x_val_scale)[:, 1]

# Evaluate model performance
con_matrix_val, outcome, pred = evaluate_model(prob_val, y_val, threshold)

# Print results for this fold
print(outcome)


### **Feature Importances from Random Forest Model**

In [ ]:
# Extract feature importances from the trained model
importances = model.feature_importances_

# Get indices of the top 20 most important features (from lowest to highest)
indices = np.argsort(importances)[-20:]

# Retrieve the original feature names
features = x.columns

# Set up the plot size
plt.subplots(figsize=(3, 7))
plt.title('Top 20 Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


### **Recursive Feature Elimination Based on Importance Ranking**
- This section performs backward feature elimination using the importance rankings derived from the random forest model. Starting from all features, we iteratively remove one feature at a time (from least to most important) and retrain the model to evaluate how performance changes on the validation set.


In [ ]:
# Get feature importances from the trained model
importances = model.feature_importances_
indices = np.argsort(importances)  # Sort features by importance (low to high)

dump(importances, rf'{cache_dir}\importances_{var}.joblib')
dump(indices, rf'{cache_dir}\index_{var}.joblib')

features = x.columns.tolist()
feature_sel = [features[i] for i in indices]
features_selected = ['0'] + feature_sel

perf_list = []

# Begin backward elimination loop
# Each iteration removes one feature (starting from the least important), retrains the model, and evaluates performance
for i in tqdm(features_selected[:-1]): 
    if i == '0':
        # Initial condition: full feature set
        features_selected = feature_sel.copy()
    else:
        # Remove the current feature from selection
        features_selected.remove(i)
    
    # Subset the selected features for training and validation
    x_train_sel = x_train[features_selected]
    x_val_sel = x_val[features_selected]

    x_train_imp, m_mean = imputer(x_train_sel, impute_method, train=True)
    x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

    x_val_imp = imputer(x_val_sel, impute_method, False, m_mean)
    x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)


    model = RandomForestClassifier(random_state=random_seed)

    weights_train = np.array([weight_for_0 if label == 0 else weight_for_1 for label in y_train])

    model.fit(x_train_scale, y_train, sample_weight=weights_train)

    prob_val = model.predict_proba(x_val_scale)[:, 1]
    con_matrix_val, outcome, pred = evaluate_model(prob_val, y_val, threshold)

    perf_list.append(outcome).


feature_select = pd.concat(perf_list).reset_index(drop=True)
# Track which feature was removed in each iteration (none removed in the first step)
feature_select['feature_removed'] = [''] + feature_sel[:-1]


In [ ]:
feature_select.head()

### **Retraining Using Best N Features Identified via Feature Selection**


In [ ]:
# Select the number of remaining features based on previous feature selection
best_number = 40  # remove n features

# Select a specific fold to reproduce best split
i = 3
print(var)

# Define target and features
y = data_train['Label']
x = data_train.loc[:, chosen_col]

# Load saved train/validation split indices
train_index = load(f'{cache_dir}\\train_index_{i}.joblib')
val_index = load(f'{cache_dir}\\val_index_{i}.joblib')

# Split dataset
x_train, x_val = x.iloc[train_index], x.iloc[val_index]
y_train, y_val = y.iloc[train_index], y.iloc[val_index]

# Retrieve full feature list and sorted importance ranking
features = x.columns.tolist()
feature_sel = [features[i] for i in indices]  # features sorted by importance

# Get the best feature subset from the selected point onward
removed_feature = feature_select.loc[best_number, 'feature_removed']
best_features = feature_sel[feature_sel.index(removed_feature) + 1:]
print(best_features)

# Prepare feature-reduced training and validation sets
x_train_best = x_train[best_features]
x_val_best = x_val[best_features]

# Impute and scale training data
x_train_imp, m_mean = imputer(x_train_best, impute_method, train=True)
x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

# Impute and scale validation data
x_val_imp = imputer(x_val_best, impute_method, False, m_mean)
x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)

# Train model 
model = RandomForestClassifier(random_state=random_seed)
weights_train = np.array([weight_for_0 if label == 0 else weight_for_1 for label in y_train])
model.fit(x_train_scale, y_train, sample_weight=weights_train)
prob_val = model.predict_proba(x_val_scale)[:, 1]

# Evaluate model performance
con_matrix_val, outcome, pred = evaluate_model(prob_val, y_val, threshold)
print(f'test_set_outcome = {outcome}')


### **Random Forest Hyperparameter Tuning using RandomizedSearchCV**


In [ ]:
# Define hyperparameter grid for Random Forest tuning
param_distributions = {
    'model__n_estimators': randint(50, 200),               # Number of trees in the forest
    'model__max_depth': randint(5, 20),                    # Maximum depth of each tree
    'model__min_samples_split': randint(2, 10),            # Min samples required to split a node
    'model__min_samples_leaf': randint(1, 5),              # Min samples required at a leaf node
    'model__criterion': ['gini', 'entropy'],               # Splitting criteria
    'model__max_features': ['sqrt', 'log2', None]          # Max features to consider at each split
}

# Define custom scoring metrics
scoring = {
    'ROC_AUC': make_scorer(roc_auc_score, needs_proba=True),
    'F1_Score': make_scorer(f1_score)
}

# Use best-selected features for training
x_train_best = x_train[best_features]

# Create a pipeline: imputation -> scaling -> model
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaling', StandardScaler()),
    ('model', RandomForestClassifier(random_state=random_seed))
])

# Randomized search with 5-fold cross-validation
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=2000,                      # Number of parameter settings to sample
    cv=5,                             # 5-fold CV
    scoring=scoring,                 
    refit='ROC_AUC',                 # Use AUROC as the primary metric for refitting
    verbose=4,
    random_state=random_seed,
    n_jobs=None                       
)

# Fit the search object
start_time = time.time()
search.fit(x_train_best, y_train, model__sample_weight=weights_train)

# Compute training time in hours
end_time = time.time()
total_time_seconds = end_time - start_time
total_time_hours = total_time_seconds / 3600
print(f"Total execution time: {total_time_hours:.2f} hours")

# Output best performance and parameters
print("Best AUROC Score:", search.best_score_)
print("Best Parameters:", search.best_params_)
print("Best Estimator:", search.best_estimator_)


### **Final Random Forest Model Training and Evaluation Across All Cohorts**


- Assign x and y

In [ ]:
y = data_train['Label']
y_test= data_test['Label']
y_ANH= data_ANH['Label']
y_WMH= data_WMH['Label']

x = data_train.loc[:, chosen_col]  
x_test= data_test.loc[:, chosen_col]
x_ANH= data_ANH.loc[:, chosen_col]
x_WMH= data_WMH.loc[:, chosen_col]

# Load saved train/validation split indices
train_index = load(f'{cache_dir}\\train_index_{i}.joblib')
val_index = load(f'{cache_dir}\\val_index_{i}.joblib')

# Split dataset
x_train, x_val = x.iloc[train_index], x.iloc[val_index]
y_train, y_val = y.iloc[train_index], y.iloc[val_index]

In [ ]:
best_number = 40 # removed feature numbers
fold = 3 # Best split

features = x.columns.tolist()
feature_sel = [features[i] for i in indices]
best_features = feature_sel[feature_sel.index(feature_select.loc[best_number,'feature_removed'])+1:] 

print (best_features)
n = len(best_features)

# Choose features with best performance
x_train_best = x_train[best_features]
x_val_best = x_val [best_features]
x_test_best = x_test [best_features]
x_ANH_best = x_ANH [best_features]
x_WMH_best = x_WMH [best_features]

# Developing set
x_train_imp, m_mean = imputer(x_train_best, impute_method, train=True)
x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

x_val_imp = imputer(x_val_best, impute_method, False, m_mean)
x_val_scale, train_scaler = scaling(x_val_imp, train_scaler, train=False)

# Internal validation set
x_test_imp = imputer(x_test_best, impute_method, False, m_mean)
x_test_scale, train_scaler = scaling(x_test_imp, train_scaler, train=False)

# ANH dataset
x_ANH_imp = imputer(x_ANH_best, impute_method, False, m_mean)
x_ANH_scale, train_scaler = scaling(x_ANH_imp, train_scaler, train=False)

# WMH dataset
x_WMH_imp = imputer(x_WMH_best, impute_method, False, m_mean)
x_WMH_scale, train_scaler = scaling(x_WMH_imp, train_scaler, train=False)

model = RandomForestClassifier(random_state=random_seed, min_samples_leaf=2, min_samples_split=2,
                               n_estimators=196, max_depth=16, criterion='entropy')

weights_train = np.array([weight_for_0 if label == 0 else weight_for_1 for label in y_train])

model.fit (x_train_scale, y_train, sample_weight=weights_train)
prob_val = model.predict_proba(x_val_scale)[:, 1]
prob_test = model.predict_proba(x_test_scale)[:, 1]
prob_ANH = model.predict_proba(x_ANH_scale)[:, 1]
prob_WMH = model.predict_proba(x_WMH_scale)[:, 1]

con_matrix_val, outcome, pred =evaluate_model ( prob_val, y_val, threshold)
con_matrix_test, outcome_test, pred_test =evaluate_model ( prob_test, y_test, threshold)
con_matrix_ANH, outcome_ANH, pred_AN =evaluate_model ( prob_ANH, y_ANH, threshold)
con_matrix_WMH, outcome_WMH, pred_WMH =evaluate_model ( prob_WMH, y_WMH, threshold)

# Store performance DataFrames
cohort_results = {
    'Developing cohort': outcome,
    'Internal validation cohort': outcome_test,
    'ANH cohort': outcome_ANH,
    'WMH cohort': outcome_WMH
}

# Calculate Brier score
brier_scores = {
    'Developing cohort': brier_score_loss(y_val, prob_val),
    'Internal validation cohort': brier_score_loss(y_test, prob_test),
    'ANH cohort': brier_score_loss(y_ANH, prob_ANH),
    'WMH cohort': brier_score_loss(y_WMH, prob_WMH)
}

# Print both performance metrics and Brier Score
for cohort_name in cohort_results:
    print(f'\n===== {cohort_name} =====')
    print(cohort_results[cohort_name].round(3).to_string(index=False))
    print(f'Brier Score: {brier_scores[cohort_name]:.3f}')

['@SD-LMALS-LY', '@SD-MALS-NE', '@SD-C-NE', '@MN-LMALS-EO', '@MN-LALS-MO', '@MN-UMALS-EO', '@MN-UMALS-NE', '@MN-MALS-NE', '@SD-LALS-EO', '@SD-V-EO', '@SD-UMALS-LY', '@SD-LMALS-EO', 'MO', 'MO#', 'BA#', '@SD-UMALS-NE', '@SD-V-NE', 'RBC', 'NE', '@SD-C-MO', '@SD-AL2-LY', 'NLR', '@WDOP', '@MN-V-LY', '@PDW', '@SD-AL2-MO', 'EO#', '@MN-V-MO', 'PLR', 'PLT', '@MN-V-EO', 'Gender', '@SD-V-LY', 'WBC', '@PCT', 'MCHC', '@SD-V-MO', '@WNOP', 'HCT', '@MAF', 'RDW', 'MCH', 'HGB', '@LHD', 'NRBC', 'MCV', 'Age']

===== Developing cohort =====
 AUROC  AUPRC  accuracy  F1_score  recall  specificity  precision   NPV
 0.947  0.832     0.913     0.756   0.717        0.958      0.799 0.936
Brier Score: 0.067

===== Internal validation cohort =====
 AUROC  AUPRC  accuracy  F1_score  recall  specificity  precision   NPV
  0.95  0.835      0.92     0.754   0.762         0.95      0.746 0.954
Brier Score: 0.061

===== ANH cohort =====
 AUROC  AUPRC  accuracy  F1_score  recall  specificity  precision   NPV
 0.948  0.83